In [ ]:
# Import all necessary libraries
import torch
import os
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model

print("All libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"Transformers version: {transformers.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")


In [ ]:
# SFT Configuration variables
# Start with a small and fast model for the first run
# This can be swapped later for a larger model like 'gpt2-medium' or 'microsoft/DialoGPT-medium'
base_model_name = 'distilgpt2'

# Dataset and output paths
dataset_path = './data/train_prefs.jsonl'
sft_output_dir = './models/sft'

print(f"Configuration:")
print(f"Base model: {base_model_name}")
print(f"Dataset path: {dataset_path}")
print(f"SFT output directory: {sft_output_dir}")

# Ensure output directory exists
os.makedirs(sft_output_dir, exist_ok=True)


In [ ]:
# Load and format the dataset for SFT training
print("Loading dataset for SFT...")

# Load the prepared dataset
train_dataset = load_dataset('json', data_files=dataset_path)['train']

print(f"Dataset loaded with {len(train_dataset)} examples")
print(f"Original keys: {list(train_dataset[0].keys())}")

def format_example(example):
    """
    Format example for SFT training by combining prompt and chosen response
    into a single text field with proper formatting.
    """
    formatted_text = f"### Human:\n{example['prompt']}\n\n### Assistant:\n{example['chosen']}"
    return {"text": formatted_text}

# Apply formatting to the entire dataset
print("\nFormatting dataset...")
formatted_dataset = train_dataset.map(format_example)

print(f"Formatted dataset keys: {list(formatted_dataset[0].keys())}")
print(f"\nExample formatted text (first 300 chars):")
print(formatted_dataset[0]['text'][:300] + "...")

print(f"\nDataset ready for SFT training with {len(formatted_dataset)} examples!")


In [ ]:
# Load base model and tokenizer
print(f"Loading tokenizer and model: {base_model_name}")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# Critical step: Check and set pad token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print("✓ Pad token set to EOS token")
else:
    print("✓ Pad token already exists")

print(f"Tokenizer loaded - Vocab size: {len(tokenizer)}")

# Load the base model
print("Loading base model...")
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float16,  # Use half precision for memory efficiency
    device_map="auto"  # Automatically place model on available GPU
)

print(f"✓ Model loaded successfully!")
print(f"Model parameters: {model.num_parameters():,}")
print(f"Model device: {next(model.parameters()).device}")

# Print model architecture summary
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")


In [ ]:
# Configure LoRA (Parameter-Efficient Fine-Tuning)
print("Configuring LoRA for efficient fine-tuning...")

# Create LoRA configuration
lora_config = LoraConfig(
    r=16,                    # Rank of the update matrices (higher = more parameters)
    lora_alpha=32,           # LoRA scaling factor (usually 2*r)
    lora_dropout=0.05,       # Dropout rate for LoRA layers
    bias="none",             # No bias updates
    task_type="CAUSAL_LM"    # Task type for causal language modeling
)

print("LoRA Configuration:")
print(f"  Rank (r): {lora_config.r}")
print(f"  Alpha: {lora_config.lora_alpha}")
print(f"  Dropout: {lora_config.lora_dropout}")
print(f"  Bias: {lora_config.bias}")
print(f"  Task type: {lora_config.task_type}")

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Print trainable parameters after applying LoRA
model.print_trainable_parameters()

print("✓ LoRA configuration complete!")


In [ ]:
# Configure training arguments optimized for RTX 4060
print("Configuring training arguments for RTX 4060...")

training_args = TrainingArguments(
    output_dir=sft_output_dir,
    per_device_train_batch_size=2,        # Small batch size for 8GB VRAM
    gradient_accumulation_steps=8,         # Effective batch size = 2*8 = 16
    learning_rate=2e-4,                    # Learning rate for LoRA
    logging_steps=20,                      # Log every 20 steps
    num_train_epochs=1,                    # Number of training epochs
    max_steps=-1,                          # Train for epochs, not steps
    bf16=True,                             # Essential for RTX 4060 performance
    save_strategy="epoch",                 # Save at the end of each epoch
    evaluation_strategy="no",              # No evaluation during training
    remove_unused_columns=False,           # Keep all dataset columns
    push_to_hub=False,                     # Don't push to Hugging Face Hub
    report_to=None,                        # Disable wandb/tensorboard logging
    dataloader_pin_memory=False,           # Reduce memory usage
    gradient_checkpointing=True,           # Trade compute for memory
)

print("Training Arguments:")
print(f"  Batch size per device: {training_args.per_device_train_batch_size}")
print(f"  Gradient accumulation steps: {training_args.gradient_accumulation_steps}")
print(f"  Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  Learning rate: {training_args.learning_rate}")
print(f"  Number of epochs: {training_args.num_train_epochs}")
print(f"  BF16 enabled: {training_args.bf16}")

# Instantiate the SFTTrainer
print("\nInitializing SFTTrainer...")

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=formatted_dataset,
    tokenizer=tokenizer,
    peft_config=lora_config,
    dataset_text_field="text",             # The column containing our formatted text
    max_seq_length=512,                    # Maximum sequence length
    packing=False,                         # Don't pack multiple samples
)

print("✅ SFTTrainer initialized successfully!")
print(f"Total training steps: {len(formatted_dataset) // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps) * training_args.num_train_epochs}")


In [ ]:
# Start training and save the model
print("🚀 Starting SFT training...")
print("This may take some time depending on your dataset size and hardware.")
print("You can monitor the progress through the logging output below.")
print("-" * 60)

# Start the training process
trainer.train()

print("-" * 60)
print("🎉 SFT training finished!")

# Save the LoRA adapters
print("\nSaving the trained model...")
trainer.save_model()

print(f"✅ Model saved successfully to: {sft_output_dir}")
print("\nThe following files have been created:")
print("  📁 adapter_config.json - LoRA configuration")
print("  📁 adapter_model.safetensors - Trained LoRA weights")
print("  📁 training_args.bin - Training arguments")

# Verify saved files
saved_files = os.listdir(sft_output_dir)
print(f"\nActual files in {sft_output_dir}:")
for file in saved_files:
    file_path = os.path.join(sft_output_dir, file)
    file_size = os.path.getsize(file_path) / (1024 * 1024)  # Size in MB
    print(f"  📄 {file} ({file_size:.2f} MB)")

print(f"\n🎯 Your LoRA adapters are ready! These small files contain all the knowledge")
print(f"needed to make the base model {base_model_name} follow your customer service style.")
print(f"\nNext step: Use these adapters in reward model training or inference!")
